In [1]:
import sys
import numpy as np
import pandas as pd
import os
import cv2
import wandb
from datetime import datetime
from tqdm import tqdm
import argparse
import random
import json
import collections

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import ConcatDataset

from sklearn.metrics import f1_score, accuracy_score

sys.path.append('/home/hong/hc701/HC701-PROJECT')
from VAL import test

import torch.backends.cudnn as cudnn

import timm
from timm.models import create_model

/home/chong.tian/.conda/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from hc701fed.dataset.dataset_list_transform import (
    MESSIDOR_binary_pairs_train,
    MESSIDOR_binary_pairs_test,
    MESSIDOR_binary_Etienne_train,
    MESSIDOR_binary_Etienne_test,
    MESSIDOR_binary_Brest_train,
    MESSIDOR_binary_Brest_test
)

centerlized_train = ConcatDataset([MESSIDOR_binary_pairs_train, MESSIDOR_binary_Etienne_train, MESSIDOR_binary_Brest_train])
centerlized_test = ConcatDataset([MESSIDOR_binary_pairs_test, MESSIDOR_binary_Etienne_test, MESSIDOR_binary_Brest_train])

horizontal_flip {'p': 0}


In [3]:
import copy
def test(model_, test_loader, device):
    model_test = copy.deepcopy(model_)
    model_test.to(device)
    model_test.eval()
    y_pred = []
    y_true = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model_test(images)
            _, predicted = torch.max(outputs.data, 1)
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    return y_true, y_pred

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 8
# Data loader
train_loader_pairs = DataLoader(MESSIDOR_binary_pairs_train, batch_size=batch_size, shuffle=True)
train_loader_Etienne = DataLoader(MESSIDOR_binary_Etienne_train, batch_size=batch_size, shuffle=True)
train_loader_Brest = DataLoader(MESSIDOR_binary_Brest_train, batch_size=batch_size, shuffle=True)
train_loader = DataLoader(centerlized_train, batch_size=batch_size, shuffle=True)

test_loader_pairs = DataLoader(MESSIDOR_binary_pairs_test, batch_size=1, shuffle=False)
test_loader_Etienne = DataLoader(MESSIDOR_binary_Etienne_test, batch_size=1, shuffle=False)
test_loader_Brest = DataLoader(MESSIDOR_binary_Brest_test, batch_size=1, shuffle=False)
test_loader = DataLoader(centerlized_test, batch_size=1, shuffle=False)

train_list = [train_loader_pairs, train_loader_Etienne, train_loader_Brest]
test_list = [test_loader_pairs, test_loader_Etienne, test_loader_Brest]

In [5]:
# centerlized train
model = create_model('vgg16', pretrained=True, num_classes=2)

model.to(device)
cudnn.benchmark = True

criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(10):
    for X, y in train_loader:
        X = X.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_pred = model(X)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()

    y_pred, y_true = test(model, test_loader, device)

    print('Epoch: {}, Loss: {:.4f}, Accuracy+f1/2: {:.4f}'.format(epoch, loss.item(), (accuracy_score(y_true, y_pred) + f1_score(y_true, y_pred, average='binary'))/2))

y_pred, y_true = test(model, test_loader, device)

print('Epoch: {}, Loss: {:.4f}, Accuracy+f1/2: {:.4f}'.format(epoch, loss.item(), (accuracy_score(y_true, y_pred) + f1_score(y_true, y_pred, average='binary'))/2))

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/chong.tian/.cache/torch/hub/checkpoints/vgg16-397923af.pth


Epoch: 0, Loss: 0.6704, Accuracy+f1/2: 0.6363
Epoch: 1, Loss: 0.4029, Accuracy+f1/2: 0.6125
Epoch: 2, Loss: 0.5121, Accuracy+f1/2: 0.6029
Epoch: 3, Loss: 0.9235, Accuracy+f1/2: 0.5306
Epoch: 4, Loss: 0.1746, Accuracy+f1/2: 0.6632
Epoch: 5, Loss: 0.6526, Accuracy+f1/2: 0.2836
Epoch: 6, Loss: 0.8615, Accuracy+f1/2: 0.6971
Epoch: 7, Loss: 0.7641, Accuracy+f1/2: 0.5498
Epoch: 8, Loss: 0.4540, Accuracy+f1/2: 0.8097
Epoch: 9, Loss: 0.6398, Accuracy+f1/2: 0.7534


In [6]:
y_pred, y_true = test(model, test_loader, device)

print('Epoch: {}, Loss: {:.4f}, Accuracy+f1/2: {:.4f}'.format(epoch, loss.item(), (accuracy_score(y_true, y_pred) + f1_score(y_true, y_pred, average='binary'))/2))

Epoch: 9, Loss: 0.6398, Accuracy+f1/2: 0.7534


In [5]:
for noise_scale in [0.1]:
    for seed in [42,43,44]:
        print('seed: {}, noise_scale: {}'.format(seed, noise_scale))
        cudnn.deterministic = True
        cudnn.benchmark = True
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        random.seed(seed)
        np.random.seed(seed)
        os.environ['PYTHONHASHSEED'] = str(seed)
        torch.manual_seed(seed)
        

        # use vgg16 as the base model
        model = create_model('vgg16', pretrained=False, num_classes=2)

        # load the fix model
        model_name = 'vgg16'
        model_path = '/home/hong/hc701/random_init/{}.pth'.format(model_name)
        model.load_state_dict(torch.load(model_path))
        mean = torch.tensor(0, dtype=torch.float)
        std = torch.tensor(0.0948*noise_scale, dtype=torch.float)
        lr=0.01
        clip_value=30

        def local_train(trains_loader_item, num_updates,model_init,client_id,rounds,lr=lr):
            __model=copy.deepcopy(model_init)
            __model.to(device)
            # Local train
            loss = torch.nn.CrossEntropyLoss()
            optimizer=torch.optim.SGD(__model.parameters(),lr=lr)
            __model.train()
            update_count = 0
            for i, (X, y) in tqdm(enumerate(trains_loader_item)):
                update_count += 1
                X, y = X.to(device), y.to(device)
                # Compute prediction and loss
                pred = __model(X)
                _loss = loss(pred,y)
                # Backpropagation
                _loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                if update_count >= num_updates:
                    break
            return __model,model_init
        
        def get_update(model_previous: torch.nn.modules,model_t: torch.nn.modules):
            updates=torch.tensor([]).to('cuda').reshape(1,-1)
            for p_t,p_pre in zip(model_t.parameters(),model_previous.parameters()):
                p_pre=p_pre.to('cuda')
                updates=torch.cat((updates,torch.flatten(p_t-p_pre).to('cuda').reshape(1,-1)),1)
            return updates
        
        def apply_dp(update_tensor,total_round,clip_threshold=clip_value,learn_rate=lr,dp_std=std,dp_mean=mean):
            # clip gradient
            updates_norm=torch.linalg.vector_norm(update_tensor)
            clip_threshold=learn_rate*clip_threshold
            update_tensor_clip=update_tensor/torch.max(torch.tensor([1]).to(device),updates_norm/clip_threshold)
            # add noise
            update_tensor_clip+=torch.normal(mean=dp_mean, std=learn_rate*dp_std*clip_threshold*total_round,size=update_tensor_clip.shape).to('cuda')
            return update_tensor_clip
        
        def update_model(dp_update_tensor: torch.Tensor(), model_pre: torch.nn.modules,model_t: torch.nn.modules):
            dp_update_tensor=torch.flatten(dp_update_tensor)  # Flatten update make it to 1D
            # Save model dict
            pre_dict_model=model_pre.state_dict()
            t_dict_model=model_t.state_dict()
            #update by the DP guarantee delta
            for name, param in model_t.named_parameters():
                length_paramter=int((torch.flatten(param).shape)[0]) # Get the length of of paramter
                delta=dp_update_tensor[:length_paramter].reshape(param.shape)  # Delta is the parameter grad times the lr
                t_dict_model[name]=pre_dict_model[name]+delta  # update model
                dp_update_tensor=dp_update_tensor[length_paramter:] # remove used
            model_t.load_state_dict(t_dict_model) # update model
            return model_t
        
        def local_dp_train(trains_loader_item, num_updates,model_init,client_id,rounds):
            model_t,model_pre=local_train(trains_loader_item=trains_loader_item, num_updates=num_updates,model_init=model_init,client_id=client_id,rounds=rounds)
            model_update=get_update(model_pre,model_t)
            # print(model_update)
            dp_update_tensor=apply_dp(update_tensor=model_update,total_round=rounds)
            # print(dp_update_tensor)
            model_t_dp=update_model(dp_update_tensor=dp_update_tensor,model_pre=model_pre,model_t=model_t)
            return model_t_dp
        
        def local_step(training_dataloaders_list,num_updates,model_init,rounds):
            MODEL_LIST=[model_init for i in range(3)]
            for i,j in zip(training_dataloaders_list,range(3)):
                MODEL_LIST[j]=local_dp_train(trains_loader_item=i,num_updates=num_updates,model_init=model_init,client_id=j,rounds=rounds)
            return MODEL_LIST
        
        def aggregation_model(models_list):
            _models_list=copy.deepcopy(models_list)
            fed_state_dict=collections.OrderedDict()
            weight_keys=models_list[0].state_dict().keys()
            for key in weight_keys:
                key_avg=0
                for _model in _models_list:
                    key_avg=key_avg+_model.state_dict()[key]*1/3
                fed_state_dict[key]=key_avg
            for _model in _models_list:
                _model.load_state_dict(fed_state_dict)
            return _models_list
        
        for com_round in [1]:
            global_models=[copy.deepcopy(model) for i in range(3)]
            for rounds in range(com_round):
                models=local_step(train_list,1000,global_models[0],com_round)
            y_true,y_pred=test(models[0],test_loader,device)
            print('Round: {} Accuracy: {}'.format(com_round,accuracy_score(y_true,y_pred)))
            print('Round:',com_round,'f1_score:',f1_score(y_true,y_pred,average='macro'))
            print('Round:',com_round,'average_Acc_f1:',(accuracy_score(y_true,y_pred)+f1_score(y_true,y_pred,average='macro'))/2)

            y_true2,y_pred2=test(models[1],test_loader,device)
            print('Round: {} Accuracy: {}'.format(com_round,accuracy_score(y_true2,y_pred2)))
            print('Round:',com_round,'f1_score:',f1_score(y_true2,y_pred2,average='macro'))
            print('Round:',com_round,'average_Acc_f1:',(accuracy_score(y_true2,y_pred2)+f1_score(y_true2,y_pred2,average='macro'))/2)

            y_true3,y_pred3=test(models[2],test_loader,device)
            print('Round: {} Accuracy: {}'.format(com_round,accuracy_score(y_true3,y_pred3)))
            print('Round:',com_round,'f1_score:',f1_score(y_true3,y_pred3,average='macro'))
            print('Round:',com_round,'average_Acc_f1:',(accuracy_score(y_true3,y_pred3)+f1_score(y_true3,y_pred3,average='macro'))/2)

seed: 42, noise_scale: 0


38it [00:03, 10.16it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [10]:
import numpy as np

np.sqrt(2*np.log(1.25/10**(-5)))

4.844805262605389